In [ ]:
#Install dependencies
import tensorflow as tf
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
import tensorboard as tb
BASE_FOLDER = '/Users/oblivisheee/Documents/GitHub/novelsdreamer-ru-t4m/'
os.chdir(BASE_FOLDER)


from modules.transformer_custom import Transformer, beam_search_decoder
from modules.regularization import RegularizedDenseLayer
from modules.create_config import transformer_config, trainer_config
from modules.tf_text_postprocess import TextPostprocessor
from modules.datagen import DataGenerator


# Config training

In [ ]:
# Configure the Transformer layer
config = transformer_config()
config_train = trainer_config()

embedding_layer = tf.keras.layers.Embedding(config["input_vocab_size"], config["d_model"])

transformer = Transformer(config["num_layers"], config["d_model"], config["num_heads"], config["dff"],
                          config["input_vocab_size"], config["input_vocab_size"], pe_input=config["maximum_position_encoding"],
                          pe_target=config["maximum_position_encoding"], rate=config["dropout_rate"], embedding=embedding_layer)

regularized_layer = RegularizedDenseLayer(config["d_model"])

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=config_train['loss_reduction'])
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=config_train['learning_rate'])

metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]

transformer.compile(optimizer=optimizer, loss=loss_object, metrics=metrics)



In [ ]:
# Configure the dataset
MAIN_DATASET_DIR = 'dataset'
TRAIN_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'train')
VALID_DATASET_DIR = os.path.join(MAIN_DATASET_DIR, 'valid')

datagen = DataGenerator(TRAIN_DATASET_DIR, VALID_DATASET_DIR)
(train_english, train_russian), (valid_english, valid_russian) = datagen.generate()

train_english = np.array(train_english)
train_russian = np.array(train_russian)
valid_english = np.array(valid_english)
valid_russian = np.array(valid_russian)

if len(train_english.shape) == 1:
    train_english = np.expand_dims(train_english, axis=-1)
if len(train_russian.shape) == 1:
    train_russian = np.expand_dims(train_russian, axis=-1)
if len(valid_english.shape) == 1:
    valid_english = np.expand_dims(valid_english, axis=-1)
if len(valid_russian.shape) == 1:
    valid_russian = np.expand_dims(valid_russian, axis=-1)

train_dataset = tf.data.Dataset.from_tensor_slices((train_english, train_russian))
validation_dataset = tf.data.Dataset.from_tensor_slices((valid_english, valid_russian))
